# Motivation
论文原文：[《Inductive Representation Learning on Large Graphs》](https://arxiv.org/abs/1706.02216)

参考学习地址：[GraphSAGE：我寻思GCN也没我牛逼](https://zhuanlan.zhihu.com/p/74242097)

GCN的基本思想：把一个节点在图中的高维度邻接信息降维到一个低维的向量表示（通过学习一个对邻居顶点进行聚合表示的函数来产生目标顶点的embedding向量）。优点可以捕捉Graph的全局信息，从而很好地表示node特征；缺点是Transductive learning方式（通常指训练与测试阶段使用相同的图结构），需要把所有节点都参与训练才能得到node embedding，对于新node的embedding获取慢。

GraphSAGE（Graph SAmple and aggreGatE）换了一种思路：既然新增的节点，一定会改变原有节点的表示，无需一定要得到每个节点的一个固定的表示，转而学习一种节点的表示方式（聚合函数，也即一个节点的信息是怎么通过其邻居节点的特征聚合而来的）。这样无论graph如何改变，都可以很容易地得到新的表示。

<img style="display: block; margin: 0 auto;" src="../../../assets/images/graphsage.png" width = "800" height = "350" alt="GraphSAGE" align=center />

## Embedding Generation
运行流程分为三个步骤：
1. 对图中每个顶点邻居顶点进行采样
2. 根据聚合函数聚合邻居顶点蕴含的信息
3. 得到图中各顶点的向量表示供下游任务使用

### 采样邻居顶点
为保证计算效率，对每个顶点采样一定数量的邻居顶点作为待聚合信息的顶点。设采样数量为k，若顶点邻居数少于k,则采用有放回的抽样方法，直到采样出k个顶点。若顶点邻居数大于k，则采用无放回的抽样

### 生成向量
<img style="display: block; margin: 0 auto;" src="../../../assets/images/graphsage-1.png" width = "600" height = "300" alt="GraphSAGE" align=center />

这里$K$是网络的层数，也代表着每个顶点能够聚合的邻接点的跳数，如$K=2$的时候每个顶点可以最多根据其2跳邻接点的信息学习其自身的embedding表示。在每一层的循环$k$中，对每个顶点$v$，首先使用$v$的邻接点的$k-1$层的embedding表示$h_u^{k-1}$来产生其邻居顶点的第$k$层聚合表示$h_{N(v)}^k$，之后将$h_{N(v)}^k$和顶点$v$的第$k-1$层表示$h_v^{k-1}$进行拼接，经过一个非线性变换产生顶点$v$的第$k$层embedding表示$h_v^k$。

### 聚合函数的选取
由于图中顶点的邻居天然无序，希望构造出的聚合函数是对称的（改变输入顺序，函数输出结果不变），同时具备较高的表达能力。
* Mean aggregator：$h_v^k \leftarrow \sigma(\mathbf{W}\cdot MEAN(\{h_v^{k-1}\} \bigcup \{h_u^{k-1}, \forall u \in N(v)\}))$，mean aggregator将目标顶点和邻居顶点的第k-1层向量拼接起来，然后对向量的每个维度进行求均值的操作，将得到的结果做一次非线性变换产生目标顶点的第k层表示向量。直接产生顶点的向量表示，而不是邻居顶点的向量表示
* Pooling aggregator：$AGGREGATE^{pool}k = max(\{\sigma (\mathbf{W}poolh_{u_i}^k+b), \forall u_i \in N(v)\})$，Pooling aggregator 先对目标顶点的邻接点表示向量进行一次非线性变换，之后进行一次pooling操作(maxpooling or meanpooling)，将得到结果与目标顶点的表示向量拼接，最后再经过一次非线性变换得到目标顶点的第k层表示向量
* LSTM aggregator：LSTM相比简单的求平均操作具有更强的表达能力，然而由于LSTM函数不是关于输入对称的，所以在使用时需要对顶点的邻居进行一次乱序操作

### 参数的学习
在定义好聚合函数之后，接下来就是对函数中的参数进行学习。文章分别介绍了无监督学习和监督学习两种方式
* 无监督：基于图的损失函数希望临近的顶点具有相似的向量表示，同时让分离的顶点的表示尽可能区分。 目标函数$J_{\mathcal{G}}(\mathbf{z}_u)=-log(\sigma (-\mathbf{z}_u^T \mathbf{z}_{v_n})) - Q \cdot \mathbb{E}_{v_n \tilde P_n(v)} log(\sigma (-\mathbf{z}_u^T \mathbf{z}_{v_n}))$，其中$v$是通过固定长度的随机游走出现在$u$附近的顶点，$p_n$是负采样的概率分布，$Q$是负样本的数量。与DeepWalk不同的是，这里的顶点表示向量是通过聚合顶点的邻接点特征产生的，而不是简单的进行一个embedding lookup操作得到
* 有监督：根据任务的不同直接设置目标函数即可，如最常用的节点分类任务使用交叉熵损失函数。